# Cleaning the Data

### Row Headings
"Vehicle Make" = brand, "Violation County" = borough

In [3]:
import pandas as pd
import numpy as np

In [17]:
# csv_filename = 'Parking_Violations_Issued_-_Fiscal_Year_2017.csv'
csv_filename = 'Parking_Violations_Issued_-_Fiscal_Year_2017_sample.csv'

# This field holds a list of all the columns that we are exploring
features = ['Plate ID', 'Issue Date', 'Violation Time', 'Issue Datetime', 'Vehicle Make', 'Vehicle Body Type', 'Violation County', 'Street Name', 'Violation Code', 'Violation Description']

df = pd.read_csv(csv_filename)
# df = df.sample(10000)

# Clean data of all null issue dates and typos in violation time
def verify_valid_violation_time(entry):
    return len(entry) == 5 and entry[0:4].isdecimal() and int(entry[0:2]) <= 12 and int(entry[2:4]) < 60 and (entry[4] == 'P' or entry[4] == 'A')
df = df[df['Issue Date'].notnull()]
df = df[df['Violation Time'].notnull()]
df = df[df['Violation Time'].apply(lambda x: verify_valid_violation_time(x))]

# Combine Issue Date and violation time to gather accurate time of each violation
def format_date(date):
    if date[0:2] == '00':
        date = date.replace('00', '12')
    return f'{date[0:2]}:{date[2:]}M'
df['Violation Time'] = df['Violation Time'].apply(lambda x: format_date(x))
df['Issue Datetime'] = pd.to_datetime(df['Issue Date'] + ' ' + df['Violation Time'])
df['Issue Date'] = pd.to_datetime(df['Issue Date'])
df['Violation Time'] = pd.to_datetime(df['Violation Time']).apply(lambda x: x.time())

# Remove all null values in the fields we are exploring
for feature in features:
    df = df[df[feature].notnull()]

# Remove Typo's in Street Name (the csv appears to sometimes have part of the intersecting street leak onto to street name)
# Typo's fixed by truncating after @ sign
df['Street Name'] = df['Street Name'].apply(lambda x: x.split('@')[0])

# Standardize ways of describing borough names, and remove typos
# Note: R appears to be Staten island bc it's Richmond County
# After Rename: BX = Bronx, BK = Brooklyn, QN = Queens, MN = Manhattan, ST = Staten Island
boroughs = {'BX': 'BX', 'BK': 'BK', 'QN': 'QN', 'NY': 'MN', 'Q': 'QN', 'K': 'BK', 'ST': 'ST', 'MN': 'MN', 'R': 'ST', 'BRONX': 'BX', 'QNS': 'QN', 'KINGS': 'BK'}
df['Violation County'] = df['Violation County'].apply(lambda x: boroughs.get(x, None))
df = df[df[feature].notnull()]


# Standardize Formatting of Violation Descriptions
df['Violation Description'] = df['Violation Description'].apply(lambda x: x.upper())
df['Violation Description Includes Code'] = df['Violation Description'].apply(lambda x: x[0].isdecimal())
# df['Violation Description'] = (df['Violation Code'].astype(str) + df['Violation Description']).where(df['Violation Description Includes Code'])
df['Violation Description'] = np.where(df['Violation Description Includes Code'],
                                       df['Violation Description'],
                                       df['Violation Code'].astype(str) + '-' + df['Violation Description'])
# display('df violation description', df['Violation Description Includes Code'])
# Picking only features we are interested in
df = df[features]
df.dtypes

/var/folders/bk/6qkvtgjx0jl5dgpwgssqx5980000gn/T/ipykernel_69265/142551722.py:25: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df['Violation Time'] = pd.to_datetime(df['Violation Time']).apply(lambda x: x.time)


Plate ID                         object
Issue Date               datetime64[ns]
Violation Time                   object
Issue Datetime           datetime64[ns]
Vehicle Make                     object
Vehicle Body Type                object
Violation County                 object
Street Name                      object
Violation Code                    int64
Violation Description            object
dtype: object

In [15]:
df.sort_values(by='Issue Datetime')

,Plate ID,Issue Date,Violation Time,Issue Datetime,Vehicle Make,Vehicle Body Type,Violation County,Street Name,Violation Code,Violation Description
10545091,AVN5883,2016-05-30,2024-06-08 04:33:00,2016-05-30 04:33:00,TOYOT,SUBN,QN,S CONDUIT AVE (E/B),7,7-FAILURE TO STOP AT RED LIGHT
10546409,DSJ2351,2016-05-31,2024-06-08 19:12:00,2016-05-31 19:12:00,TOYOT,4DSD,QN,QUEENS BLVD (E/B),7,7-FAILURE TO STOP AT RED LIGHT
10550247,GAM6691,2016-06-01,2024-06-08 07:26:00,2016-06-01 07:26:00,CHEVR,SUBN,QN,SB MAIN ST,5,5-BUS LANE VIOLATION
10549097,FGS9898,2016-06-01,2024-06-08 07:43:00,2016-06-01 07:43:00,HONDA,SUBN,ST,NB HYLAN BLVD,5,5-BUS LANE VIOLATION
10548555,EFA9808,2016-06-01,2024-06-08 09:56:00,2016-06-01 09:56:00,CADIL,SUBN,QN,BEACH CHANNEL DR (W/,7,7-FAILURE TO STOP AT RED LIGHT
...,...,...,...,...,...,...,...,...,...,...
807448,VNL4097,2017-06-27,2024-06-08 16:23:00,2017-06-27 16:23:00,TOYOT,4DSD,BX,Dreiser Loop,38,38-FAILURE TO DISPLAY MUNI REC
1513628,Z41ESJ,2017-06-27,2024-06-08 16:38:00,2017-06-27 16:38:00,TOYOT,4DSD,BK,Nostrand Ave,14,14-NO STANDING
7703251,HLR7078,2017-06-27,2024-06-08 17:48:00,2017-06-27 17:48:00,INFIN,4DSD,BK,Hillel Pl,38,38-FAILURE TO DISPLAY MUNI REC
8207930,BP4182,2017-06-27,2024-06-08 18:31:00,2017-06-27 18:31:00,PREVO,BUS,MN,E 35th St,31,31-NO STAND (COM. MTR. ZONE)


In [ ]:
csv_filename = 'Parking_Violations_Issued_-_Fiscal_Year_2017.csv'
df = pd.read_csv(csv_filename)

# Find number of 
display(df['Violation County'].value_counts())
# df = df[['Violation County']]
# df.describe()